In [1]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 138 kB 1.9 MB/s eta 0:00:01


In [8]:
# automatically reload external modules if they change
%load_ext autoreload
%autoreload 2

!pip install torchvision
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchbearer
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchbearer import Trial
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import numpy as np
np.random.seed(seed)

In [10]:
train_data = pd.read_csv('../Data/input_data_crim.csv', header = None)
target = pd.read_csv('../Data/target_crim.csv', header = None)
trainloader = DataLoader(train_data, batch_size=128, shuffle=True)

In [ ]:
class SimulationData(Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target 
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, idx):
        

In [12]:

class FeedForward(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, num_classes):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size_1) 
        self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.fc3 = nn.Linear(hidden_size_2, num_classes)  
        
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        out = F.softmax(out, dim=1)
        return out

In [0]:
model = FeedForward(train_data.shape[1], 200, 200, target.shape[1])

loss_function = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters())

device = "cuda:0" if torch.cuda.is_available() else "cpu"

trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=15)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)



{'test_loss': 1.4997789859771729, 'test_acc': 0.9611999988555908}


In [0]:

class FeedForwardSplit(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, hidden_size_3, num_classes_behaviour, num_classes_target):
    super(FeedForwardSplit, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size_1) 
    self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)
    self.fc3 = nn.Linear(hidden_size_2, hidden_size_3) 
    self.fc_behaviour = nn.Linear(hidden_size_3, num_classes_behaviour) 
    self.fc_target = nn.Linear(hidden_size_3, num_classes_target) 

  def forward(self, x):
    # First layers - normal feedforward
    out = self.fc1(x)
    out = F.relu(out)
    out = self.fc2(out)
    out = F.relu(out)

    # Split network --> each have a layer before their output layers, take input from fc2 
    out_behaviour = self.fc3(out)
    out_behaviour = F.relu(out_behaviour)
    out_target = self.fc3(out)
    out_target = F.relu(out_target)

    # output layers
    out_behaviour = self.fc_behaviour(out_behaviour)
    out_target = self.fc_target(out_target)
    
    # Get softmax from output layers
    out_behaviour = F.softmax(out_behaviour, dim=1)
    out_target = F.softmax(out_target,dim=1)
    
    # Return softmax outputs as an array
    return [out_behaviour,out_target]

In [0]:

class FeedForwardCombinedOutput(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, num_classes):
        super(FeedForwardCombinedOutput, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size_1) 
        self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.fc3 = nn.Linear(hidden_size_2, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        out = F.softmax(out, dim=1)
        return out

In [0]:

class FeedForwardDropout(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, num_classes):
        super(FeedForwardDropout, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size_1) 
        self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.fc3 = nn.Linear(hidden_size_2, num_classes) 
        self.drop = nn.Dropout(0.2) 
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        out = F.softmax(out, dim=1)
        return out